In [3]:
# Import necessary libraries
import openai
import gsearch
import json
import os
import re
import tiktoken
import time
import inspect
import callgpt
import react_chain
import compliance_checker
import excel_analysis
import repo_reader
from openpyxl import load_workbook
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from tkinter import filedialog
from tkinter import *

def count_tokens(text):
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(text))
    return num_tokens
    
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()
os.environ["OPENAI_API_KEY"] = open_file('Keys/openai_api_key.txt')
openai.api_key = open_file('Keys/openai_api_key.txt')
openai_api_key = openai.api_key
callgpt = callgpt.Ask()
class BusinessAnalyst:
    def __init__(self):
        self.role_system = {"role": "system", "content": "You are a highly organised and detail oriented Goldman Sachs business analyst."}
        self.messages = [self.role_system]
        # INSERT INFO ABOUT SAVING THE PROMPT + COMPLETION PAIRS - FAISS? CHROMA?

    def chat_with_gpt3(self, prompt, max_retries=3, delay=2):
        user_message = {"role": "user", "content": prompt}
        new_message_tokens = count_tokens(prompt)
        total_tokens = sum(count_tokens(m["content"])
                        for m in self.messages) + new_message_tokens

        while total_tokens + 1 > 3500:
            # Remove the oldest user-assistant message pair to stay within the token limit
            self.messages.pop(1)
            self.messages.pop(1)
            total_tokens = sum(count_tokens(m["content"])
                            for m in self.messages) + new_message_tokens

        self.messages.append(user_message)
        response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=self.messages
                )

        # Retry logic
        retries = 0
        while retries < max_retries:
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=self.messages,
                )
                assistant_message = {
                    "role": "assistant", "content": response.choices[0].message["content"]}
                self.messages.append(assistant_message)
                return assistant_message["content"]
            except openai.error.OpenAIError as e:
                print(f"Error occurred during API call: {e}")
                retries += 1
                if retries < max_retries:
                    time.sleep(delay)  # Delay between retries
                else:
                    raise  # Raise the exception if all retries have failed


    def process_input(self, user_query):
        prompt = f"You are an AI who wants to answer the following user query: '{user_query}' with a numbered list of tasks. Response: "
        processed_query = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Processed query:*****\n" + "\033[0m\033[0m"+processed_query)
        return processed_query

    # Not strictly speaking necessary, unless we want to do nested tasks
    def decompose_tasks(self, processed_query): 
        prompt = f"You are an AI whose job is to split the following processed query into sub-tasks and create a new numbered task list: '{processed_query}'. Response: "
        sub_tasks = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Subtasks*****\n"+"\033[0m\033[0m"+ (sub_tasks))
        return sub_tasks

    def select_task(self, sub_tasks):
        prompt = f"Select the first task to analyse from '{sub_tasks}'. Response: "
        selected_task = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Selected Tasks:*****\n"+"\033[0m\033[0m"+ (selected_task))
        return selected_task

    def execute_tasks(self, selected_task, max_retries=3):
        results = []
        # ADD MORE FUNCTIONS
        # function_mapping = {
        #     "google_search": gsearch.execute
        #     # "wiki_search": react_chain.query
        #     # "excel": ba.analyse_excel,
        #     # "repo reader": repo_reader.main
        #     # Add more function mappings here
        # }

        # retries = 0
        # while retries < max_retries:
        #     best_tool = self.choose_best_tool(selected_task)
        #     if best_tool in function_mapping:
        #         tool_function = function_mapping[best_tool]
        #         break
        #     else:
        #         print(f"Invalid tool selected: {best_tool}. Retrying...")
        #         retries += 1

        # if retries == max_retries:
        #     raise ValueError("Failed to select a valid tool after multiple retries.")

        # prompt = f"{best_tool}: {selected_task}"
        # # result = react_chain.query(compliant_prompt) # Primarily google searches and wiki searches
        result = gsearch.execute(prompt)  # Use the selected function to solve the task
        print("\033[92m\033[1m" + "\n*****Results Of Execution*****\n" + "\033[0m\033[0m" + str(result))
        # result = self.chat_with_gpt3(prompt)
        return result

    # def choose_best_tool(self, task):
    #     prompt = f"You are an AI that has to select the best tool to use to solve the following task: '{task}'. Please respond in the format 'Action: [tool_name] only from this list - Action: (google_search|wiki_search|calculate. The response should contain absolutely no other words. The default option should be 'Action: google_search' "
    #     response = self.chat_with_gpt3(prompt)
    #     print(f"GPT-3 Response: {response}")  # Print the response from GPT-3
        
    #     tool_re = re.compile(r"'\s*(.*?)\s*'")
    #     tool_match = tool_re.search(response)
    #     if not tool_match:
    #         # Try to match the response without single quotes
    #         tool_re = re.compile(r"Action:\s*(google_search|wiki_search|google\ssearch)", re.IGNORECASE)
    #         tool_match = tool_re.search(response)

    #     if tool_match:
    #         best_tool = tool_match.group(1)
    #         print(f"Extracted tool from GPT-3 response: {best_tool}")  # Print the extracted tool
    #         return best_tool.strip().lower().replace(" ", "_")
    #     else:
    #         print(f"Unexpected GPT-3 response: {response}")  # Add this line to print the unexpected response
    #         raise ValueError("GPT-3 did not provide a valid tool name in its response.")
        
    def analyze_results(self, results):
        prompt = f"You are an AI focused on analysing the results from executing the task: '{results}'. Please critically explain and appraise the output. "
        analysis = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Analysis*****\n"+"\033[0m\033[0m" + (analysis))
        return analysis

    def refine_tasks(self, analysis):
        prompt = f"Considering the results, let's refine and create a new list of tasks based on the following analysis: {analysis}. The response should be in a numbered list with one sentence each. Response: "
        new_tasks = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Refined new Tasks*****\n"+"\033[0m\033[0m"+ (new_tasks))
        return new_tasks

    def generate_output(self, analysis):
        prompt = f"You are an analyst who will create a readable document from the whole discussion summarising it and clearly articulating key theories, ideas and actions to be done. Please generate a user-friendly output based on the following analysis: {analysis}. Response: "
        output = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Generated Output*****\n"+"\033[0m\033[0m"+ (output))
        return output

    def run_analysis(self, user_query):
        processed_query = self.process_input(user_query)
        sub_tasks = self.decompose_tasks(processed_query)
        selected_tasks = self.select_task(sub_tasks)
        results = self.execute_tasks(selected_tasks)
        analysis = self.analyze_results(results)
        new_tasks = self.refine_tasks(analysis)
        ques = input("Do you want to continue - Y or N? ")
        while ques == "Y":
            sub_tasks = self.execute_tasks(new_tasks)
            selected_tasks = self.select_task(sub_tasks)
            results = self.execute_tasks(selected_tasks)
            analysis = self.analyze_results(results)
            new_tasks = self.refine_tasks(analysis)
            ques = input("Do you want to continue - Y or N? ")
        if ques == "N":
            print("\n\n*******Alright! Final results coming up.*******\n\n")
        else:
            print("\n\n*******Dude! I said Y or N!*******\n\n")
        output = self.generate_output(analysis)
        return output
    
    def get_excel_file_path(self):
        root = Tk()
        root.withdraw()  # Hide the Tkinter root window
        file_path = filedialog.askopenfilename(initialdir="/", title="Select a File", filetypes=[("Excel Files", "*.xlsx")])
        print(file_path)
        root.destroy()  # Close the Tkinter root window
        return file_path

    def analyse_excel(self, prompt):
        print("\033[92m\033[1m" + "\n*****Excel Analysis*****\n" + "\033[0m\033[0m")
        functions_list = [func for func in inspect.getmembers(excel_analysis, inspect.isfunction)]

        filepath = self.get_excel_file_path()
        results = excel_analysis.main(filepath)

        for recursion_count in range(5):
            analysis = self.analyze_results(results)
            # Process the best result and create tasks
            prompt = f"As a top inventory analyst at Shopify or a similar institution, when presented with an Excel sheet functioning as an inventory management system, what tests and hypotheses would you employ to determine if there is adequate stock remaining for specific SKUs? {analysis}. Write your response as a numbered task list: "
            output = self.chat_with_gpt3(prompt)

            # Evaluate the output
            evaluation_prompt = f"How good is the following analysis: '{output}'. Please rate it on a scale of 1 to 10, with 10 being the best: "
            results = self.decompose_tasks(output)

        sub_tasks = self.process_input(results)

        final_results = []
        for task in sub_tasks:
            result = self.execute_tasks(task)
            final_results.append(result)

        # Combine the results and analyze them
        final_result = ' '.join(final_results)
        analysis = self.analyze_results(final_result)

        return final_result


ba = BusinessAnalyst()
result = None

while not result:
    user_query = input("""Choose one. What would you like to do? \n
                   1. Ask something you're curious about, run tasks \n
                   2. Figure out something about your docs \n
                   3. Figure out what's inside a python repo\n
                   4. Simulate a dialogue between two people \n
                   5. Let's write a report together! \n
                   6. Analyse an excel! \n
                   7. Write a short script. \n
                   00. Exit the prompt \n
                   """)
    if user_query == "1":
        prompt = input("Enter your query: ")
        result = ba.run_analysis(prompt)
    elif user_query == "2":
        # Call the function to figure out something about your docs
        # Replace this line with the appropriate function call
        result = "\n\nFigure out something about your docs (function not implemented)\n\n"
    elif user_query == "3":
        # Call the function to figure out what's inside a python repo
        # Replace this line with the appropriate function call
        result = "\n\nFigure out what's inside a python repo (function not implemented)\n\n"
    elif user_query == "4":
        # Call the function to simulate a dialogue between two people
        import agents_discussion
        result = agents_discussion.generate_output(user_query)
        print("\n\nHere's a dialogue!\n\n")
    elif user_query == "5":
        # Call the function to write a report together
        import agents_report
        result = agents_report.create_joint_letter(user_query)
        print("\n\nHere's the report!\n\n")
    elif user_query == "6":
        # Call the function to analyze an excel
        result = ba.analyse_excel("Analyze an excel sheet")
    elif user_query == "7":
        # Call the function to write a short script
        # Replace this line with the appropriate function call
        result = "\n\nWrite a short script (function not implemented)\n\n"
    elif user_query == "00":
        result = "\n\nSorry to see you go !!\n\n"
    else:
        print("\n\nInvalid option. Please choose a number between 1 and 7.\n\n")

if result:
    print(result)



*****Processed query:*****
Sure, I can help you with that. Here are the steps to analyze an AI startup that's trying to raise funds:

1. Start by examining the AI startup’s technology, services, and products to see how they differentiate themselves from competitors in the market.

2. Look at the company's management team and see if they have the necessary experience, skills, and track record to execute their business plan.

3. Analyze the AI startup's financials, including revenue streams, growth prospects, and cost structures, to get a sense of the company's financial health.

4. Review the market opportunity for the AI startup, including demand for their services or products, and their potential for growth and scalability.

5. Investigate the company's competition, including the competitive landscape, market share, and barriers to entry, to determine how well-positioned they are to compete.

6. Evaluate the AI startup's customer base, including the demographics, customer acquisition